In [ ]:
#load ehr_stream and target_values from pickle files
import pickle
from configuration import FilePaths
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize
import nltk
FILE_PATHS = FilePaths()

ehr_stream = pickle.load(open(FILE_PATHS.EHR_STREAM, "rb"))
target_values = pickle.load(open(FILE_PATHS.TARGET_VALUES, "rb"))

In [ ]:
df = pd.DataFrame(ehr_stream)

# Sample 

import pprint
pprint.pprint(f'Input timestream {df.loc[1290]}')
pprint.pprint(f'Will they revisit: {target_values[1290]}')

In [ ]:
def tokenize(column):
    """Tokenizes a Pandas dataframe column and returns a list of tokens.

    Args:
        column: Pandas dataframe column (i.e. df['text']).

    Returns:
        tokens (list): Tokenized list, i.e. [Donald, Trump, tweets]

    """

    tokens = nltk.word_tokenize(column)
    return [w for w in tokens]   

In [ ]:
def remove_stopwords(tokenized_column):
    """Return a list of tokens with English stopwords removed. 

    Args:
        column: Pandas dataframe column of tokenized data from tokenize()

    Returns:
        tokens (list): Tokenized list with stopwords removed.

    """
    stops = set(stopwords.words("english"))
    return [word for word in tokenized_column if not word in stops]

In [ ]:
def rejoin_words(tokenized_column):
    """Rejoins a tokenized word list into a single string. 
    
    Args:
        tokenized_column (list): Tokenized column of words. 
        
    Returns:
        string: Single string of untokenized words. 
    """
    return ( " ".join(tokenized_column))

In [ ]:
# loop through dataframe columns and tokenize
for column in df.columns:
    df[column] = df[column].apply(tokenize)
    df[column] = df[column].apply(remove_stopwords)
    df[column] = df[column].apply(rejoin_words)


In [ ]:
df['targets'] = target_values

#issue with pads returning true for all targets
df

In [ ]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(df.index,target_values,test_size=0.2)
df.iloc[X_train]
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2)

In [ ]:
# Save the train, test, and validation sets to pickle files

pickle.dump(df.iloc[X_train].values.tolist(), open(FILE_PATHS.X_TRAIN, "wb"))
pickle.dump(df.iloc[X_test].values.tolist(), open(FILE_PATHS.X_TEST, "wb"))
pickle.dump(df.iloc[X_val].values.tolist(), open(FILE_PATHS.X_VAL, "wb"))
pickle.dump(y_train, open(FILE_PATHS.Y_TRAIN, "wb"))
pickle.dump(y_test, open(FILE_PATHS.Y_TEST, "wb"))
pickle.dump(y_val, open(FILE_PATHS.Y_VAL, "wb"))